# Simple Chatbot with OpenAI API and Gradio

In this practical, we will create a simple chatbot by leveraging the capabilities of a Large Language Model (LLM) and building a user-friendly interface with Gradio. The chatbot will process user inputs, generate responses using LLM, and display them in real time.

#Load Libraries

In [ ]:
!pip install -q gradio
!pip install openai
!pip install langchain openai tiktoken chromadb python-dotenv langchain_community
!pip install U langchain_openai
!pip install python-dotenv

In [ ]:
import openai
openai.__version__

Download the Openai key



In [ ]:
import gdown
url = 'https://drive.google.com/file/d/1U43HPiy3dOLAZNZcw6TNxWmVw4MwM_4F/view?usp=drive_link'
output_path = '.env'
gdown.download(url, output_path, quiet=False,fuzzy=True)

Test the ChatGPT with the following questions



*   Which is the largest country by area in the world?




In [ ]:
import os
from dotenv import load_dotenv

# load .env file to environment
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(OPENAI_API_KEY)
AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
print(AZURE_ENDPOINT)
DEPLOYMENT_NAME = os.getenv('DEPLOYMENT_NAME')
print(DEPLOYMENT_NAME)
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
print(OPENAI_API_VERSION)


In [ ]:
from openai import AzureOpenAI

AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DEPLOYMENT_NAME = os.getenv('DEPLOYMENT_NAME')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')


from openai import AzureOpenAI


client = AzureOpenAI(
    api_key=OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
    azure_endpoint=AZURE_ENDPOINT)


def get_completion_from_messages(messages, model=DEPLOYMENT_NAME, temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

print(get_completion_from_messages([{'role':'user', 'content':'Which is the largest country by area in the world?'}]))

# LLM Chatbot

Create a LLM  Chatbot Class with function to to receive a prompt. The output is the respond from the prompt.

In [ ]:
from typing import Optional


class Chat:

    def __init__(self, system: Optional[str] = None):
        self.system = system
        self.messages = []

        if system is not None:
            self.messages.append({
                "role": "system",
                "content": system
            })

    def prompt(self, content: str) -> str:
          self.messages.append({
              "role": "user",
              "content": content
          })
          response = get_completion_from_messages(self.messages)


          self.messages.append({
              "role": "assistant",
              "content": response
          })
          return response

# System prompt

Create a system prompt to tune the LLM to be a Chatbot to collect order from customer.

In the system prompt instruct the LLM to be OrderBot receive the order from the costumer. In the instruction also provide the menu for the food and the prices.

In [ ]:

system_prompt= f"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friedly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everithing is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burguer  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""
print(system_prompt)

#Build Chatbot UI with Gradio

Create a respond function with the Chat Class. In the respond function receive the customer query and pass into the Chat object to generate the respond.

The Chat object is initialise with the system prompt as part of the context information. It is also save into the chat history. The multi-turn conversation between chatbot and customer will be append to the chat history. This provide some context during the conversation between the chatbot and customer.

In [ ]:
import gradio as gr



chat = Chat(system= str(system_prompt))


def respond(message, chat_history):
    bot_message = chat.prompt(content=message)
    chat_history.append((message, bot_message))
    return "", chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)